In [1]:
import great_expectations as gx
import pandas as pd
from datetime import datetime
from great_expectations.checkpoint import Checkpoint
from great_expectations.data_context.types.base import DataContextConfig

In [16]:
def validate(path, df):
    context = gx.get_context()
    now = datetime.now()

    datasource  = context.sources.add_pandas("house_datasource")
    data_asset = datasource.add_dataframe_asset(name="house_asset")
    batch_request = data_asset.build_batch_request(dataframe=df)

    context.add_or_update_expectation_suite(expectation_suite_name='expectation_house')
    validator = context.get_validator(
        batch_request=data_asset.build_batch_request(dataframe=df),
        expectation_suite_name='expectation_house',
    )
    
    validator.expect_column_values_to_not_be_null('TotRmsAbvGrd')
    validator.expect_column_values_to_be_between('TotRmsAbvGrd', min_value=1, max_value=30)
    validator.expect_column_values_to_not_be_null('WoodDeckSF')
    validator.expect_column_values_to_not_be_null('YrSold')
    validator.expect_column_values_to_not_be_null('1stFlrSF')
    validator.expect_column_values_to_not_be_null('Foundation')
    validator.expect_column_values_to_be_in_set(column='Foundation', value_set = [
        'BrkTil',
        'CBlock',
        'PConc',
        'Slab',
        'Stone',
        'Wood',
    ])
    validator.expect_column_values_to_not_be_null('KitchenQual')
    validator.expect_column_values_to_be_in_set(column='KitchenQual', value_set = [
        'Ex',
        'Gd',
        'TA',
        'Fa',
        'Po',
    ])
    validator.save_expectation_suite(discard_failed_expectations=False)

    checkpoint = Checkpoint(
        name="house_checkpoint",
        run_name_template="housing_checkpoint",
        data_context=context,
        batch_request=batch_request,
        expectation_suite_name='expectation_house',
        action_list=[
            {
                "name": "store_validation_result",
                "action": {"class_name": "StoreValidationResultAction"},
            },
                {"name": "update_data_docs", "action": {"class_name": "UpdateDataDocsAction"}},
            ],
        )

    context.add_or_update_checkpoint(checkpoint=checkpoint)
    result_format: dict = {
        "result_format": "COMPLETE",
        "unexpected_index_column_names": ["Id"],
    }
    return checkpoint.run(result_format=result_format)

In [22]:
def log_failed_expectations(result_json):
    run_results = result_json['run_results']
    val_result_id = list(run_results)[0]
    val_results = run_results[val_result_id]['validation_result']

    info = ""
    
    failed_rows = []
    failed_cols = []
    failed_conf = []
    
    for res in val_results['results']:
        if res['success'] == False:
            _col = res['expectation_config']['kwargs']['column'] 
            _exp_conf = res['expectation_config']['expectation_type']
            info += f'column -> {_col}\n'
            info += f'    expectation_config -> {_exp_conf}\n'
            indexes_list = res['result']['partial_unexpected_index_list']
            failed_rows_id = [item['Id'] for item in indexes_list]
            failed_rows = list(set(failed_rows + failed_rows_id))
            failed_conf = list(set(failed_conf + [_exp_conf]))
            failed_cols = list(set(failed_cols + [_col]))
            info += f'    rows with errors -> {failed_rows_id}\n'

    desc = f'Validation Error on Rows {failed_rows} of file _FILENAME_ !'
    desc += f'Columns {failed_cols} failed on the following expectations : {failed_conf}'
    
    return desc, failed_rows

In [23]:
# path = '../data/folder_C/_file_0.csv'
path = '../data/folder_A/file_2.csv'

df = pd.read_csv(path)
result = validate(path,df)
log,failed_rows = log_failed_expectations(result.to_json_dict())
print(log)
print(failed_rows)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/48 [00:00<?, ?it/s]

Validation Error on Rows [1035, 1364, 1383] of file _FILENAME_ !Columns ['KitchenQual'] failed on the following expectations : ['expect_column_values_to_not_be_null']
[1035, 1364, 1383]


In [5]:
filtered_df = df[~df['Id'].isin(failed_rows)]

rows = list(filtered_df['Id'].values)
rows

[1331, 1120, 1129, 792, 283, 876, 1448, 921]

In [7]:
path = '../data/folder_A/file_1.csv'
context = gx.get_context()
df = pd.read_csv(path)
result = validate(path,df)
log, failed_rows = log_failed_expectations(result.to_json_dict())
print(log)
print(failed_rows)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/43 [00:00<?, ?it/s]

column -> Foundation
    expectation_config -> expect_column_values_to_be_in_set
    rows with errors -> [1164, 30, 1417, 1012, 1061, 474, 571, 114, 421, 554]

[1061, 421, 1417, 554, 1164, 114, 1012, 474, 571, 30]
